In this we are adding the regularization term.

See the Andrew Ng notes on the regularization unit added.

L2 regularization : we take the square of the thetas 

L1 regularization : just take the mod of the theta values


The whole idea is to penalise the model if the values of theta re too much and it leads to overfitting of the model

In [1]:

import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn import datasets, svm
from sklearn.datasets import load_iris

In [2]:
df = pd.read_csv("../Datasets/Melbourne_housing_FULL.csv")
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


In [20]:
df.nunique()

Suburb             351
Address          34009
Rooms               12
Type                 3
Price             2871
Method               9
SellerG            388
Date                78
Distance           215
Postcode           211
Bedroom2            15
Bathroom            11
Car                 15
Landsize          1684
BuildingArea       740
YearBuilt          160
CouncilArea         33
Lattitude        13402
Longtitude       14524
Regionname           8
Propertycount      342
dtype: int64

In [21]:
# let's use limited columns which makes more sense for serving our purpose
cols_to_use = ['Suburb', 'Rooms', 'Type', 'Method', 'SellerG', 'Regionname', 'Propertycount', 
               'Distance', 'CouncilArea', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'Price']
df = df[cols_to_use]

In [23]:
df.isna().sum()

Suburb               0
Rooms                0
Type                 0
Method               0
SellerG              0
Regionname           3
Propertycount        3
Distance             1
CouncilArea          3
Bedroom2          8217
Bathroom          8226
Car               8728
Landsize         11810
BuildingArea     21115
Price             7610
dtype: int64

In [25]:
cols_to_fill_zero = ['Propertycount', 'Distance', 'Bedroom2', 'Bathroom', 'Car']

df[cols_to_fill_zero] = df[cols_to_fill_zero].fillna(0)

df['Landsize'].fillna(df['Landsize'].mean(),inplace=True)
df['BuildingArea'].fillna(df['BuildingArea'].mean(),inplace=True)

In [26]:
df.dropna(inplace=True)
## only na values are now with price and lest drop those now

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27244 entries, 1 to 34856
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         27244 non-null  object 
 1   Rooms          27244 non-null  int64  
 2   Type           27244 non-null  object 
 3   Method         27244 non-null  object 
 4   SellerG        27244 non-null  object 
 5   Regionname     27244 non-null  object 
 6   Propertycount  27244 non-null  float64
 7   Distance       27244 non-null  float64
 8   CouncilArea    27244 non-null  object 
 9   Bedroom2       27244 non-null  float64
 10  Bathroom       27244 non-null  float64
 11  Car            27244 non-null  float64
 12  Landsize       27244 non-null  float64
 13  BuildingArea   27244 non-null  float64
 14  Price          27244 non-null  float64
dtypes: float64(8), int64(1), object(6)
memory usage: 3.3+ MB


In [28]:
## lets hot encode the categorical features
df = pd.get_dummies(df,drop_first=True)
df.head()

,Rooms,Propertycount,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,Price,Suburb_Aberfeldie,...,CouncilArea_Moorabool Shire Council,CouncilArea_Moreland City Council,CouncilArea_Nillumbik Shire Council,CouncilArea_Port Phillip City Council,CouncilArea_Stonnington City Council,CouncilArea_Whitehorse City Council,CouncilArea_Whittlesea City Council,CouncilArea_Wyndham City Council,CouncilArea_Yarra City Council,CouncilArea_Yarra Ranges Shire Council
1,2,4019.0,2.5,2.0,1.0,1.0,202.0,160.2564,1480000.0,0,...,0,0,0,0,0,0,0,0,1,0
2,2,4019.0,2.5,2.0,1.0,0.0,156.0,79.0000,1035000.0,0,...,0,0,0,0,0,0,0,0,1,0
4,3,4019.0,2.5,3.0,2.0,0.0,134.0,150.0000,1465000.0,0,...,0,0,0,0,0,0,0,0,1,0
5,3,4019.0,2.5,3.0,2.0,1.0,94.0,160.2564,850000.0,0,...,0,0,0,0,0,0,0,0,1,0
6,4,4019.0,2.5,3.0,1.0,2.0,120.0,142.0000,1600000.0,0,...,0,0,0,0,0,0,0,0,1,0


In [33]:
X = df.drop("Price",axis=1)
y = df.Price

In [38]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.3, random_state=2)

In [39]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train,y_train)

print(lin_reg.score(X_train,y_train))
lin_reg.score(X_test,y_test)

## fitting best on training dataset
## but worst on testing dataset
## so it means it has acheived overfiting

0.6827792395792723


0.13853683161484953

In [41]:
## since it has overfitted
## lets use lasso regression that is l1 regression


## tol is how close you want the final value to be
from sklearn import linear_model
lasso_reg = linear_model.Lasso(alpha = 50,max_iter =100,tol = 0.1)
lasso_reg.fit(X_train, y_train)

lasso_reg.score(X_test,y_test)

## see the accuracy score on the final model got increased drascticallly

D:\Anaconda\envs\gpu\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1326631542478296.5, tolerance: 796258688308215.5
  positive)


0.6636111369404488

In [42]:
## now lets use ridge : that is L2 regularization

from sklearn import linear_model
ridge_reg = linear_model.Ridge(alpha = 50,max_iter =100,tol = 0.1)
ridge_reg.fit(X_train, y_train)

ridge_reg.score(X_test,y_test)

0.6670848945194958